In [1]:
import os
import numpy as np
import sys
import h5py
import pandas as pd
from IPython.display import display
from random import choice

ROOT_DIR = '../'
sys.path.insert(0, os.path.abspath(f'/{ROOT_DIR}/heart_rhythm_analysis/'))    # make repo root importable

from heart_rhythm_analysis.get_data.CapnoBaseETL import CapnoBaseETL
from heart_rhythm_analysis.get_data.MimicETL import MimicETL
from heart_rhythm_analysis.utils.timeseries_viewer import make_window_figure, create_time_series_viewer

bSetUpDB = True
WINDOW_LEN = 8

import dash
app = dash.Dash(__name__)

# Create Datasets from Databases

In [2]:
def load_as_df(file_path, filename,
               store_signals=True,
               group_by_record=False):
    h5_path = f"{file_path}/{filename}.h5"
    print(f'Loading: {h5_path}')
    hf = h5py.File(h5_path, "r")
    rows = []
    for subj in hf.keys():
        subj_grp = hf[subj]
        # print(subj)
        window_count = 0
        for win_id in subj_grp.keys():
            window_count = window_count+1
            win_grp = subj_grp[win_id]
            # pull out everything you need
            rec_id      = win_grp.attrs['rec_id']
            label       = win_grp.attrs['label']
            raw_ppg     = win_grp["raw_ppg"][:]
            proc_ppg    = win_grp["proc_ppg"][:]
            raw_ekg     = win_grp["raw_ekg"][:]
            raw_abp     = win_grp["raw_abp"][:]
            raw_ppg_fs  = win_grp.attrs["raw_ppg_fs"]
            ekg_fs      = win_grp.attrs["ekg_fs"]
            ppg_fs      = win_grp.attrs["ppg_fs"]
            abp_fs      = win_grp.attrs["abp_fs"]
            notes       = win_grp.attrs.get("notes", "")
            # build the window row
            row = {
                "subject": subj,
                "window_id": win_id,
                "window_count": window_count,
                "rec_id": rec_id,
                "label": label,
                "raw_ppg_fs": raw_ppg_fs,
                "ppg_fs_out": ppg_fs,
                "ekg_fs_out": ekg_fs,
                "abp_fs_out": abp_fs,
                "raw_len": len(raw_ppg),
                "proc_len": len(proc_ppg),
                "duration_raw_s": len(raw_ppg) / ppg_fs,
                "duration_proc_s": len(proc_ppg) / ppg_fs,
                "notes": notes
            }
            if store_signals:
                row.update({
                    "raw_ppg": raw_ppg,
                    "proc_ppg": proc_ppg,
                    "raw_ekg": raw_ekg,
                    "raw_abp": raw_abp
                })
            rows.append(row)

    df = pd.DataFrame(rows)

    if group_by_record and store_signals:
        # define how to aggregate each column
        agg_dict = {
            "subject":     "first",
            "label":       "first",
            "raw_ppg_fs":  "first",
            "ppg_fs_out":  "first",
            "ekg_fs_out":  "first",
            "abp_fs_out":  "first",
            "notes":       lambda x: list(x),
            "raw_len":     lambda x: np.sum(x),
            "proc_len":    lambda x: np.sum(x),
            "duration_raw_s":  lambda x: np.sum(x),
            "duration_proc_s": lambda x: np.sum(x),
            # now the key: concatenate all windows into one long array
            "raw_ppg":  lambda series: np.concatenate(series.values),
            "proc_ppg": lambda series: np.concatenate(series.values),
            "raw_ekg":  lambda series: np.concatenate(series.values),
            "raw_abp":  lambda series: np.concatenate(series.values),
        }
        df = (
            df
            .groupby("rec_id", as_index=False)
            .agg(agg_dict)
        )
    return df

## Create Capno Dataset

### CapnoBase

In [ ]:
def main():
    root_path = os.path.join('../data/raw/capnobase/data/mat')
    out_path = os.path.join('../data/processed/length_full/capnobase_db')
    out_filename = 'capnobase_db'
    if not os.path.exists(out_path):
        os.mkdir(out_path)

    fs_in = 100.00
    fs_out = 100.00

    config = {
    "input_dir"      : root_path,
    "output_dir"     : out_path,
    "window_size_sec": 30,
    "fs_in"          : fs_in,
    "fs_out"   : fs_out,
    "lowpass_cutoff" : (fs_out / 2),
    "fir_numtaps"    : 129,
    "zero_phase"     : True,
    "out_filename" :  out_filename
}
    if bSetUpDB:
        etl = CapnoBaseETL(config)
        h5file = etl.process_all()
        print(f"Saved windows HDF5 to {h5file}")
    df = load_as_df(out_path,out_filename)

    display(df.head(10))
    return df
if __name__ == "__main__":
    df_capnobase = main()

## MIMIC III

### MIMIC III AF DATASET

In [ ]:

# def main():
#     root_path = os.path.join('../data/raw/mimic_af_nonaf/mimic_af_data.mat')
#     out_filename = 'mimic_af_db'
#     out_path = os.path.join('../data/processed/length_full/mimic_af_db')
    
#     if not os.path.exists(out_path):
#         os.mkdir(out_path)
#     config = {
#         "input_dir": root_path,
#         "output_dir":  out_path,
#         "fs_in": 125.00,
#         "fs_out": 125.00,
#         "window_size_sec": 30,
#         "scale_type": "norm",
#         "zero_phase": True,
#         "decimate_signal": False,
#         "out_filename": out_filename
#     }
#     # if bSetUpDB:
#     #     etl = MimicETL(config)
#     #     out_file = etl.process()
#     #     print("Saved AF windows to", out_file)
#     df = load_as_df(out_path,out_filename)
#     display(df.head(10))
#     return df

# if __name__ == "__main__":
#     df_mimc_af = main()
    

### MIMIC III Non-AF Dataset

In [ ]:

# def main():
#     root_path = os.path.join('../data/raw/mimic_af_nonaf/mimic_non_af_data.mat')
#     out_filename = 'mimic_non_af_db'
#     out_path = os.path.join('../data/processed/length_full/mimic_non_af_db')
    
#     if not os.path.exists(out_path):
#         os.mkdir(out_path)
#     config = {
#         "input_dir": root_path,
#         "output_dir":  out_path,
#         "fs_in": 125.00,
#         "fs_out": 125.00,
#         "window_size_sec": 30,
#         "scale_type": None,
#         "decimate_signal": False,
#         "zero_phase": True,
#         "out_filename": out_filename   
#     }
#     if bSetUpDB:
#         etl = MimicETL(config)
#         out_file = etl.process()
#         print("Saved non AF windows to", out_file)
#     df = load_as_df(out_path,out_filename)
#     display(df.head(10))
#     return df
# if __name__ == "__main__":
#     df_mimc_nonaf = main()

### MIMIC III General Subset

In [3]:
def main():
    root_path = os.path.join('../data/raw/mimic3_data/mimic3_struct.mat')
    out_filename = 'mimic3_db'
    out_path = os.path.join('../data/processed/length_full/mimic3_db')
    
    if not os.path.exists(out_path):
        os.mkdir(out_path)
    config = {
        "input_dir": root_path,
        "output_dir":  out_path,
        "fs_in": 125.00,
        "fs_out": 20.83,
        'fs_ekg': 125,
        'fs_bp':125,
        "window_size_sec": WINDOW_LEN,
        "scale_type": "norm",
        "decimate_signal": True,
        "zero_phase": True,
        "out_filename": out_filename 
    }
    bSetUpDB = True
    if bSetUpDB:
        etl = MimicETL(config)
        out_file = etl.process()
        print("Saved General MIMIC III windows to", out_file)
    df = load_as_df(out_path,out_filename)
    # display(df.head(10))
    return df
if __name__ == "__main__":
    df_mimic3_gen = main()
    # first = df_mimic3_gen.iloc[np.random.randint(0, len(df_mimic3_gen))] 
    chosen_subj = df_mimic3_gen['subject'].unique()[0]

    surr_subject_df = df_mimic3_gen[df_mimic3_gen['subject'] == chosen_subj]

    display(surr_subject_df.head())
    initial_window = df_mimic3_gen.sample(1).iloc[0]

    specs = [
        {"key":"raw_ppg",  "fs_key":"raw_ppg_fs",  "subplot":1, "legend":"Raw PPG"},
        {"key":"proc_ppg", "fs_key":"ppg_fs_out",  "subplot":1, "legend":"Proc PPG"},
        {"key":"raw_ekg",  "fs_key":"ekg_fs_out",  "subplot":2,               },
        {"key":"raw_abp",  "fs_key":"abp_fs_out",  "subplot":3,               },
    ]

    fig_fn = lambda window: make_window_figure(window, specs)
    
    app = create_time_series_viewer(
    surr_subject_df,
    fig_fn,
    specs,
    index_label_fn=lambda idx, row: f"{row.subject}-{row.window_count}"
)
    print(f'Total # of Subjects: {len(df_mimic3_gen['subject'].unique())}')
    print(f'Total # of Windows: {df_mimic3_gen.shape[0]}')
    print("Launching Dash at http://127.0.0.1:8050 …")
    app.run(debug=True)

Loading ../data/raw/mimic3_data/mimic3_struct.mat
Saved General MIMIC III windows to ../data/processed/length_full/mimic3_db/mimic3_db.h5
Loading: ../data/processed/length_full/mimic3_db/mimic3_db.h5


,subject,window_id,window_count,rec_id,label,raw_ppg_fs,ppg_fs_out,ekg_fs_out,abp_fs_out,raw_len,proc_len,duration_raw_s,duration_proc_s,notes,raw_ppg,proc_ppg,raw_ekg,raw_abp
0,p000160,00391907-3835-4bf2-b455-90bdb9276251,1,3531764,-1,125.0,20.83,125,125,1000,167,48.007681,8.017283,,"[0.4946236559139785, 0.4946236559139785, 0.494...","[-0.82902277, 0.17097723, -0.067515075, 0.0629...","[0.5901960784313726, 0.6, 0.596078431372549, 0...","[105.99746153680672, 103.06395245368478, 100.9..."
1,p000160,0052578a-691e-4515-813e-933bcce603cd,2,3531764,-1,125.0,20.83,125,125,1000,167,48.007681,8.017283,,"[0.5386119257086999, 0.5376344086021505, 0.535...","[-0.73288304, 0.26711696, 0.0057798624, 0.1491...","[0.6254901960784314, 0.6294117647058823, 0.641...","[64.34163255647493, 69.2308143616782, 77.24907..."
2,p000160,00786f40-6c22-4a1c-a067-30940e2b0b9e,3,3531764,-1,125.0,20.83,125,125,1000,167,48.007681,8.017283,,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[-0.48947465, -0.48910898, -0.4909103, -0.4738...","[0.5745098039215686, 0.5901960784313726, 0.605...","[98.37033792068964, 98.37033792068964, 98.5659..."
3,p000160,00c5ea91-70c6-43d4-ae6b-104809683849,4,3531764,-1,125.0,20.83,125,125,1000,167,48.007681,8.017283,,"[0.49266862170087977, 0.49266862170087977, 0.4...","[-0.837578, 0.162422, -0.07661563, 0.08459675,...","[0.5843137254901961, 0.6, 0.6, 0.5647058823529...","[135.72368691244256, 135.9192541846507, 135.91..."
4,p000160,011b7029-20b8-4a7e-8f7c-099c94c18852,5,3531764,-1,125.0,20.83,125,125,1000,167,48.007681,8.017283,,"[0.5483870967741935, 0.552297165200391, 0.5571...","[-0.55547786, 0.44452214, 0.25203043, 0.372302...","[0.5647058823529412, 0.5901960784313726, 0.574...","[103.45508699810104, 101.890548820436, 99.9348..."


Total # of Subjects: 100
Total # of Windows: 45000
Launching Dash at http://127.0.0.1:8050 …


Address already in use
Port 8050 is in use by another program. Either identify and stop that program, or start the server with a different port.


SystemExit: 1

/Users/shayanriyaz/.local/lib/python3.12/site-packages/IPython/core/interactiveshell.py:3587: UserWarning:

To exit: use 'exit', 'quit', or Ctrl-D.



## MIMIC IV

### MIMIC IV General Data

In [4]:
np.random.seed(42)
def main():
    substring = ''
    file_name = f'_{substring}' if len(substring) > 0 else ""
    root_path = os.path.join(f'../data/raw/mimic4_data/mimic4{file_name}_struct.mat')
    out_filename = f'mimic4{file_name}_db'
    out_path = os.path.join(f'../data/processed/length_full/{out_filename}')

    if not os.path.exists(out_path):
        os.mkdir(out_path)
    fs_in = 62.5
    fs_out = 20.83
    config = {
        "input_dir": root_path,
        "output_dir":  out_path,
        "fs_in": fs_in,
        "fs_out": fs_out,
        'fs_ekg': 62.5, 
        'fs_bp':fs_in,
        "window_size_sec": WINDOW_LEN,
        "scale_type": "norm",
        "decimate_signal": True,
        "zero_phase": True,
        "out_filename": out_filename 
    }
    bSetUpDB = True
    if bSetUpDB:
        etl = MimicETL(config)
        out_file = etl.process()
        print(f"Saved {substring} MIMIC IV  windows to", out_file)
    df = load_as_df(out_path,out_filename)
    return df

if __name__ == "__main__":

    df_mimic4_gen = main()
    first = df_mimic4_gen.iloc[np.random.randint(0, len(df_mimic4_gen))] 
    initial_window = df_mimic4_gen.sample(1).iloc[0]
    specs = [
        {"key":"raw_ppg",  "fs_key":"raw_ppg_fs",  "subplot":1, "legend":"Raw PPG"},
        {"key":"proc_ppg", "fs_key":"ppg_fs_out",  "subplot":1, "legend":"Proc PPG"},
        {"key":"raw_ekg",  "fs_key":"ekg_fs_out",  "subplot":2,               },
        {"key":"raw_abp",  "fs_key":"abp_fs_out",  "subplot":3,               },
    ]

    # 3) bind specs into a single-arg figure fn
    fig_fn = lambda window: make_window_figure(window, specs)
    
    app = create_time_series_viewer(
    df_mimic4_gen,
    fig_fn,
    specs,
    index_label_fn=lambda idx, row: f"{row.subject}-{row.window_count}"
)
    print(f'Total # of Subjects: {len(df_mimic4_gen['subject'].unique())}')
    print(f'Total # of Windows: {df_mimic4_gen.shape[0]}')
    print("Launching Dash at http://127.0.0.1:8050 …")
    
    # app.run(debug=True)
    

Loading ../data/raw/mimic4_data/mimic4_struct.mat
Saved  MIMIC IV  windows to ../data/processed/length_full/mimic4_db/mimic4_db.h5
Loading: ../data/processed/length_full/mimic4_db/mimic4_db.h5
Total # of Subjects: 51
Total # of Windows: 22899
Launching Dash at http://127.0.0.1:8050 …


In [5]:
subject_agg = {
    # sampling & metadata: just take the first (they're constant per subject)
    "raw_ppg_fs": "first",
    "ppg_fs_out": "first",
    "ekg_fs_out": "first",
    "abp_fs_out": "first",
    # lengths & durations: sum across windows
    "raw_len":       "sum",
    "proc_len":      "sum",
    "duration_raw_s":  "sum",
    "duration_proc_s": "sum",
    # signals: concatenate all windows end-to-end
    "raw_ppg":  lambda s: np.concatenate(s.values),
    "proc_ppg": lambda s: np.concatenate(s.values),
    "raw_ekg":  lambda s: np.concatenate(s.values),
    "raw_abp":  lambda s: np.concatenate(s.values),
    # if you want to keep track of window_ids or rec_ids:
    "window_id": lambda s: list(s.values),
    "rec_id":    lambda s: list(s.values),
    "label":     lambda s: list(s.values),
}

# ── 2) group by subject ─
df_by_subject = (
    df_mimic4_gen
    .groupby("subject", as_index=False)
    .agg(subject_agg)
)

# ── 3) now df_by_subject has one row per subject, with each signal a long array
print(df_by_subject.shape)
print(df_by_subject.raw_ppg.iloc[0].shape)  # e.g. (sum of all its windows,) 

# bind specs & figure fn exactly as before
fig_fn = lambda row: make_window_figure(row, specs)

# subject‐level dashboard (labels are just the subject IDs)
app = create_time_series_viewer(
    df_by_subject,
    fig_fn,
    specs,
    index_label_fn=lambda idx, row: str(row.subject)
)
print("Launching Dash at http://127.0.0.1:8080 …")
app.run(debug=True,port=8080)

(51, 16)
(224051,)
Launching Dash at http://127.0.0.1:8080 …


Address already in use
Port 8080 is in use by another program. Either identify and stop that program, or start the server with a different port.


SystemExit: 1

/Users/shayanriyaz/.local/lib/python3.12/site-packages/IPython/core/interactiveshell.py:3587: UserWarning:

To exit: use 'exit', 'quit', or Ctrl-D.



# Create Train/Test Files

In [6]:
from heart_rhythm_analysis.get_data.prepare_dataset import my_split_and_save

ROOT_PATH = '../data/processed/length_full'
file_paths = [f"{ROOT_PATH}/mimic3_db/mimic3_db.h5", f"{ROOT_PATH}/mimic4_db/mimic4_db.h5"]
out_path = '../data/development_dataset/length_full'
train_path, test_path = my_split_and_save(file_paths, train_ratio=0.8, output_dir=out_path)

def count_windows(h5_path):
    with h5py.File(h5_path, 'r') as f:
        subjects = list(f.keys())
        n_subjects = len(subjects)
        n_windows  = sum(len(f[subj].keys()) for subj in subjects)
    return n_subjects, n_windows

# Replace these with your real paths:
paths = {
    "mimic3": "../data/processed/length_full/mimic3_db/mimic3_db.h5",
    "mimic4": "../data/processed/length_full/mimic4_db/mimic4_db.h5",
    "train":   "../data/development_dataset/length_full/train_dataset.h5",
    "test":    "../data/development_dataset/length_full/test_dataset.h5",
}

for name, p in paths.items():
    if os.path.exists(p):
        subs, wins = count_windows(p)
        print(f"{name:>6} → {subs} subjects, {wins} windows")
    else:
        print(f"{name:>6} → file not found: {p}")

mimic3 → 100 subjects, 45000 windows
mimic4 → 51 subjects, 22899 windows
 train → 151 subjects, 54319 windows
  test → 151 subjects, 13580 windows


In [ ]:
np.concatenate(df_mimic4_gen['raw_ppg'].to_list())